See email from Jim 23/03/2016 10:32 for details of VR-track setup

#Plan
- Create various setups (see Dushi email 02/10/2015 16:47)
    - vrpipe-setup —based_on
        - pf_pdna_import
        - pf3kgatk_mapping
    - wait for Dushi before running pf3kgatk_combine_gvcfs
- output to /lustre/scratch109/malaria/WillH_1
- final output to /nfs/team112_internal/production/release_build/Pf/WillH_1



In [2]:
%run standard_imports.ipynb

python 3.4.3 |Anaconda 2.2.0 (64-bit)| (default, Mar  6 2015, 12:03:53) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
numpy 1.9.2
scipy 0.15.1
pandas 0.15.2
numexpr 2.3.1
pysam 0.8.3
petl 1.0.11
petlx 1.0.3
vcf 0.6.7
h5py 2.4.0
tables 3.1.1
vcfplt 0.8


In [3]:
RELEASE_DIR = '/nfs/team112_internal/production/release_build/Pf/WillH_1'
RESOURCES_DIR = '%s/resources' % RELEASE_DIR

GENOME_FN = "/lustre/scratch109/malaria/pf3k_methods/resources/Pfalciparum.genome.fasta"
SNPEFF_DIR = "/lustre/scratch109/malaria/pf3k_methods/resources/snpEff"
REGIONS_FN = "/nfs/team112_internal/rp7/src/github/malariagen/pf-crosses/meta/regions-20130225.bed.gz"

RELEASE_METADATA_FN = "%s/Pf_WillH_1_sample_metadata.txt" % RELEASE_DIR
WG_VCF_FN = "%s/vcf/Pf_WillH_1.vcf.gz" % RELEASE_DIR

BCFTOOLS = '/nfs/team112_internal/rp7/src/github/malariagen/methods-dev/pf3k_techbm/opt_4/bcftools/bcftools'


In [4]:
print(WG_VCF_FN)

/nfs/team112_internal/production/release_build/Pf/WillH_1/vcf/Pf_WillH_1.vcf.gz


In [5]:
chromosomes = ["Pf3D7_%02d_v3" % x for x in range(1, 15, 1)] + [
    'Pf3D7_API_v3', 'Pf_M76611'
]
chromosome_vcfs = ["%s/vcf/SNP_INDEL_%s.combined.filtered.vcf.gz" % (RELEASE_DIR, x) for x in chromosomes]
# print(chromosome_vcfs)


In [4]:
if not os.path.exists(RESOURCES_DIR):
    os.makedirs(RESOURCES_DIR)

In [8]:
!cp {GENOME_FN}* {RESOURCES_DIR}
!cp -R {SNPEFF_DIR} {RESOURCES_DIR}
!cp -R {REGIONS_FN} {RESOURCES_DIR}

# VRTrack setup (Jim)

# Import

#Mapping

#Mark duplicates

#Indel realignment

#Base quality score recalibration (BQSR)

#Merge lanes

#Merge libraries

#Sample bam summaries

#Symlinks for sample bams

#HaplotypeCaller

#Combine gvcfs

#Genotype gvcfs

#Variant recalibration SNPs

#Variant recalibration indels

#Apply recalibration SNPs

#Apply recalibration indels

#Annotation SNPs

#Annotation indels

#Combine SNPs and indels

#Filter variants

#Symlinks for VCFs

# Create whole genome vcf

In [18]:
# Reran this as below piping through sed command to fix AD in header
# !{BCFTOOLS} concat --output {WG_VCF_FN} --output-type z {" ".join(chromosome_vcfs)}
# !tabix -p vcf {WG_VCF_FN}

In [6]:
!{BCFTOOLS} concat {" ".join(chromosome_vcfs)} \
| sed 's/##FORMAT=<ID=AD,Number=./##FORMAT=<ID=AD,Number=R/' \
| bgzip -c > {WG_VCF_FN}
!tabix -p vcf {WG_VCF_FN}

[tabix] the index file exists. Please use '-f' to overwrite.


# Create bam summary file

In [33]:
!vrpipe-fileinfo \
--setup WillH_mergelibs \
--step 1 \
--metadata sample,study,bases_callable_percent,bases_no_coverage_percent,bases_low_coverage_percent,bases_poor_mapping_quality_percent,mean_coverage \
> {RELEASE_METADATA_FN}


# Create README section containing variant numbers

In [8]:
number_of_snps = !{BCFTOOLS} query -f '%CHROM\t%POS\n' --include 'TYPE="snp"' {WG_VCF_FN} | wc -l
number_of_indels = !{BCFTOOLS} query -f '%CHROM\t%POS\n' --include 'TYPE!="snp"' {WG_VCF_FN} | wc -l
number_of_pass_snps = !{BCFTOOLS} query -f '%CHROM\t%POS\n' --include 'FILTER="PASS" && TYPE="snp"' {WG_VCF_FN} | wc -l
number_of_pass_indels = !{BCFTOOLS} query -f '%CHROM\t%POS\n' --include 'FILTER="PASS" && TYPE!="snp"' {WG_VCF_FN} | wc -l
number_of_pass_biallelic_snps = !{BCFTOOLS} query -f '%CHROM\t%POS\n' --include 'FILTER="PASS" && TYPE="snp" && N_ALT=1' {WG_VCF_FN} | wc -l
number_of_pass_biallelic_indels = !{BCFTOOLS} query -f '%CHROM\t%POS\n' --include 'FILTER="PASS" && TYPE!="snp" && N_ALT=1' {WG_VCF_FN} | wc -l
number_of_VQSLODgt6_snps = !{BCFTOOLS} query -f '%CHROM\t%POS\n' --include 'FILTER="PASS" && TYPE="snp" && VQSLOD>6' {WG_VCF_FN} | wc -l
number_of_VQSLODgt6_indels = !{BCFTOOLS} query -f '%CHROM\t%POS\n' --include 'FILTER="PASS" && TYPE!="snp" && VQSLOD>6' {WG_VCF_FN} | wc -l

print("%s variants" % ("{:,}".format(int(number_of_snps[0]) + int(number_of_indels[0]))))
print("%s SNPs" % ("{:,}".format(int(number_of_snps[0]))))
print("%s indels" % ("{:,}".format(int(number_of_indels[0]))))
print()
print("%s PASS variants" % ("{:,}".format(int(number_of_pass_snps[0]) + int(number_of_pass_indels[0]))))
print("%s PASS SNPs" % ("{:,}".format(int(number_of_pass_snps[0]))))
print("%s PASS indels" % ("{:,}".format(int(number_of_pass_indels[0]))))
print()
print("%s PASS biallelic variants" % ("{:,}".format(int(number_of_pass_biallelic_snps[0]) + int(number_of_pass_biallelic_indels[0]))))
print("%s PASS biallelic SNPs" % ("{:,}".format(int(number_of_pass_biallelic_snps[0]))))
print("%s PASS biallelic indels" % ("{:,}".format(int(number_of_pass_biallelic_indels[0]))))
print()
print("%s VQSLOD>6.0 variants" % ("{:,}".format(int(number_of_VQSLODgt6_snps[0]) + int(number_of_VQSLODgt6_indels[0]))))
print("%s VQSLOD>6.0 SNPs" % ("{:,}".format(int(number_of_VQSLODgt6_snps[0]))))
print("%s VQSLOD>6.0 indels" % ("{:,}".format(int(number_of_VQSLODgt6_indels[0]))))
print()
     

583,970 variants
346,940 SNPs
237,030 indels

156,912 PASS variants
57,728 PASS SNPs
99,184 PASS indels

90,361 PASS biallelic variants
50,142 PASS biallelic SNPs
40,219 PASS biallelic indels

66,057 VQSLOD>6.0 variants
32,869 VQSLOD>6.0 SNPs
33,188 VQSLOD>6.0 indels



In [9]:
print('''
The VCF file contains details of {number_of_variants} discovered variants of which {number_of_snps}
are SNPs and {number_of_indels} are indels (or multi-allelic mixtures of SNPs
and indels). It is important to note that many of these variants are
considered low quality. Only the variants for which the FILTER column is set
to PASS should be considered of high quality. There are {number_of_pass_variants} such high-
quality PASS variants ({number_of_pass_snps} SNPs and {number_of_pass_indels} indels).

The FILTER column is based on two types of information. Firstly certain regions
of the genome are considered "non-core". This includes sub-telomeric regions,
centromeres and internal VAR gene clusters on chromosomes 4, 6, 7, 8 and 12.
The apicoplast and mitochondrion are also considered non-core. All variants within
non-core regions are considered to be low quality, and hence will not have the
FILTER column set to PASS. The regions which are core and non-core can be found
in the file resources/regions-20130225.bed.gz.

Secondly, variants are filtered out based on a quality score called VQSLOD. All
variants with a VQSLOD score below 0 are filtered out, i.e. will have a value of
Low_VQSLOD in the FILTER column, rather than PASS. The VQSLOD score for each
variant can be found in the INFO field of the VCF file. It is possible to use the
VQSLOD score to define a more or less stringent set of variants. For example for
a very stringent set of the highest quality variants, select only those variants
where VQSLOD >= 6. There are {number_of_VQSLODgt6_snps} such stringent SNPs and {number_of_VQSLODgt6_indels}
such stringent indels.

It is also important to note that some variants have more than two alleles. For
example, amongst the {number_of_pass_snps} high quality PASS SNPs, {number_of_pass_biallelic_snps} are biallelic. The
remaining {number_of_pass_multiallelic_snps} high quality PASS SNPs have 3 or more alleles. Similarly, amongst
the {number_of_pass_indels} high-quality PASS indels, {number_of_pass_biallelic_indels} are biallelic. The remaining
{number_of_pass_multiallelic_indels} high quality PASS indels have 3 or more alleles.
'''.format(
        number_of_variants="{:,}".format(int(number_of_snps[0]) + int(number_of_indels[0])),
        number_of_snps="{:,}".format(int(number_of_snps[0])),
        number_of_indels="{:,}".format(int(number_of_indels[0])),
        number_of_pass_variants="{:,}".format(int(number_of_pass_snps[0]) + int(number_of_pass_indels[0])),
        number_of_pass_snps="{:,}".format(int(number_of_pass_snps[0])),
        number_of_pass_indels="{:,}".format(int(number_of_pass_indels[0])),
        number_of_VQSLODgt6_snps="{:,}".format(int(number_of_VQSLODgt6_snps[0])),
        number_of_VQSLODgt6_indels="{:,}".format(int(number_of_VQSLODgt6_indels[0])),
        number_of_pass_biallelic_snps="{:,}".format(int(number_of_pass_biallelic_snps[0])),
        number_of_pass_biallelic_indels="{:,}".format(int(number_of_pass_biallelic_indels[0])),
        number_of_pass_multiallelic_snps="{:,}".format(int(number_of_pass_snps[0]) - int(number_of_pass_biallelic_snps[0])),
        number_of_pass_multiallelic_indels="{:,}".format(int(number_of_pass_indels[0]) - int(number_of_pass_biallelic_indels[0])),
    )
)


The VCF file contains details of 583,970 discovered variants of which 346,940
are SNPs and 237,030 are indels (or multi-allelic mixtures of SNPs
and indels). It is important to note that many of these variants are
considered low quality. Only the variants for which the FILTER column is set
to PASS should be considered of high quality. There are 156,912 such high-
quality PASS variants (57,728 SNPs and 99,184 indels).

The FILTER column is based on two types of information. Firstly certain regions
of the genome are considered "non-core". This includes sub-telomeric regions,
centromeres and internal VAR gene clusters on chromosomes 4, 6, 7, 8 and 12.
The apicoplast and mitochondrion are also considered non-core. All variants within
non-core regions are considered to be low quality, and hence will not have the
FILTER column set to PASS. The regions which are core and non-core can be found
in the file resources/regions-20130225.bed.gz.

Secondly, variants are filtered out based on a quali

In [12]:
"{number_of_pass_variants}/{number_of_variants} variants ({pct_pass}%) pass all filters".format(
        number_of_variants="{:,}".format(int(number_of_snps[0]) + int(number_of_indels[0])),
        number_of_pass_variants="{:,}".format(int(number_of_pass_snps[0]) + int(number_of_pass_indels[0])),
        pct_pass=round((
            (int(number_of_pass_snps[0]) + int(number_of_pass_indels[0])) /
            (int(number_of_snps[0]) + int(number_of_indels[0]))
        ) * 100)
)

'156,912/583,970 variants (27%) pass all filters'

# Make all files read-only

In [13]:
!chmod -R uga-w {RELEASE_DIR}

chmod: changing permissions of `/nfs/team112_internal/production/release_build/Pf/WillH_1': Operation not permitted
chmod: changing permissions of `/nfs/team112_internal/production/release_build/Pf/WillH_1/Antoine_samples_vrpipe2.txt': Operation not permitted
chmod: changing permissions of `/nfs/team112_internal/production/release_build/Pf/WillH_1/Antoine_samples_meta.tab': Operation not permitted


# Ignore

In [1]:
# %run standard_imports.ipynb

python 3.4.3 |Anaconda 2.2.0 (64-bit)| (default, Mar  6 2015, 12:03:53) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
numpy 1.9.2
scipy 0.15.1
pandas 0.15.2
numexpr 2.3.1
pysam 0.8.3
petl 1.0.11
petlx 1.0.3
vcf 0.6.7
h5py 2.4.0
tables 3.1.1
vcfplt 0.8
